# Otimização: Gradiente Descendente

*Descrição:*

Implementar o método do gradiente com algumas escolhas de *tamanho de passo* e utilizá-lo para minimizar funções da forma: 

$$f(x) = \sum\limits_{i=1}^na_i x_i^2$$

para diferentes valores de $ a_i > 0 $.

### **Funções Auxiliares**

Vamos definir algumas funções que serão utilizadas posteriormente para facilitar os cálculos do *método gradiente*.

In [1]:
import numpy as np

gradient = lambda x,a: 2*x*a # retorna lista com os gradientes dado ponto (x,y)
apply_func = lambda x,a: np.dot(np.square(x), a) # f(x)
close_to_zero = lambda grad, tol: np.linalg.norm(grad,2) < tol # stop condition (euclidian distance)
armijo = lambda x, grad, step, a, alpha: apply_func(x - (grad*step),a) >= apply_func(x,a) + alpha*step*np.dot(grad, -1*grad)

### Função Principal

Agora já conseguimos definir a função *gradient_descent* (método gradiente). Esta função possui 2 modos de execução:

- **Passo Fixo:** Para `backtracking=False`, utilizamos o *tamanho de passo* (`step_size`) de maneira fixa, ou seja, seu valor não será alterado com a execução do algoritmo.
- **Passo com Backtracking:** Para `backtracking=True`, utilizamos a estratégia de *busca linear* e, assim, o valor do passo será decrescido de acordo com a [regra de Armijo](https://en.wikipedia.org/wiki/Wolfe_conditions#Armijo_rule_and_curvature)

In [2]:
def gradient_descent(x, a, step_size, backtracking=True, tolerance=0.01, alpha=0.3, verbose=False):
    
    grad = gradient(x,a)
    it = 0

    while((close_to_zero(grad,tolerance) == False)):
        
        # Choose good step_size
        step = step_size
        if(backtracking == True): 
            while(armijo(x, grad, step, a, alpha) == True):
                step = step / 2 
        
        # Update stuff
        x = x - (grad*step)
        grad = gradient(x,a)
        it = it + 1
        
    if(verbose == True):
        print('Iteracoes utilizadas = ', it)
        
    return(x)

Exemplo de uso:   

In [3]:
a = np.float64([1, 2, 3]) # temos, por exemplo, esses valores de a > 0
x = np.float64([2,2,2]) # e um chute inicial dos valores de xi

convergence_x = gradient_descent(x, a, step_size=1, backtracking=True, tolerance=0.001, alpha=0.1, verbose=True)
print("Ponto de mínimo encontrado: ", convergence_x)

Iteracoes utilizadas =  14
Ponto de mínimo encontrado:  [0.00012207 0.         0.00012207]


##  Encontrando o passo constante ótimo.

Para determinar o passo ótimo do algoritmo para uma função  da forma $f(x) = \sum\limits_{i=1}^na_i x_i^2$, podemos utilizar a constante de [lipschitz](https://en.wikipedia.org/wiki/Lipschitz_continuity).

Dada essa função, temos que os autovalores da sua Hessiana (é uma matriz diagonal)  são os elementos da diagonal principal e assumem a forma $2a_i$ (segunda derivada de $a_ix_i^2$) . Assim, considerando $\mu$ o seu menor autovalor e $L$ o seu maior autovalor conseguimos garantir a convergência do método com passo fixo quando temos um $t < 2/L$.

Porém, como queremos minimizar o termo $max\{|1-t\mu|,|1-tL|\}$, há um $t_{ótimo}$ que garante a melhor taxa de convergência e ele se dá por: 

<center>$t_{ótimo} = \frac{2}{(L + \mu)}$<center>

Tais informações serão utilizadas para execução do exercício a seguir:

In [4]:
get_mu = lambda a: 2*min(a)
get_L = lambda a: 2*max(a)

Vamos calcular  $\mu$ e $L$ para a = [1, 1.1].

In [5]:
a = [1, 1.1]
mu, L = get_mu(a), get_L(a)

print('mu = ', mu)
print('L = ', L)

mu =  2
L =  2.2


Temos que a convergência é garantida quando $t < 2/L$ e o passo ótimo $t_{ótimo} = \frac{2}{(L + \mu)}$: 

In [6]:
optimal_step_1 = 2/(L+mu)

print('convergencia garantida se t < ', 2/L)
print('passo otimo t = ', 2/(L+mu))

convergencia garantida se t <  0.9090909090909091
passo otimo t =  0.47619047619047616


O mesmo processo pode ser realizado para obter o valor de convergência garantida e passo ótimo para a função com os valores de $a_i$ seguintes: a = [1,100]

In [7]:
a = [1, 100]
mu2, L2 = get_mu(a), get_L(a)

print('mu = ', mu2)
print('L = ', L2)

mu =  2
L =  200


In [8]:
optimal_step_2 = 2/(L2+mu2)

print('convergencia garantida se t < ', 2/L2)
print('passo otimo t = ', 2/(L2+mu2))

convergencia garantida se t <  0.01
passo otimo t =  0.009900990099009901


## Experimentos

### Testes Simples ($n = 2$)

Para padronizar os experimentos, utilizaremos $x_{inicial} = (1,2)$. 

**i) Passo constante: t ótimo**

Função: $f(x_1, x_2) = 1x_1^2 + 1.1x_2^2$

In [9]:
a = np.float64([1,1.1]) # temos, por exemplo, esses valores de a > 0
x = np.float64([1,2]) # e um chute inicial dos valores de xi

convergence_x = gradient_descent(x, a, step_size=optimal_step_1, backtracking=False, tolerance=0.001, verbose=True)
print("Ponto de mínimo encontrado: ", convergence_x)

Iteracoes utilizadas =  3
Ponto de mínimo encontrado:  [ 0.00010798 -0.00021596]


Função: $f(x_1, x_2) = 1x_1^2 + 100x_2^2$

In [10]:
a = np.float64([1,100]) # temos, por exemplo, esses valores de a > 0
x = np.float64([1,2]) # e um chute inicial dos valores de xi

convergence_x = gradient_descent(x, a, step_size=optimal_step_2, backtracking=False, tolerance=0.001, verbose=True)
print("Ponto de mínimo encontrado: ", convergence_x)

Iteracoes utilizadas =  645
Ponto de mínimo encontrado:  [ 2.49697633e-06 -4.99395266e-06]


**Análise:** Assim, temos a convergência para valores próximos a zero em ambos os exemplos. Em particular, obseva-se que no primeiro exemplo a convergência é significativamente mais rápida (requer menos iterações).

**ii) Passo constante: t não-ótimo**

Escolhemos valores não ótimos, porém que garantam a convergência, como calculado anteriormente.

Função: $f(x_1, x_2) = 1x_1^2 + 1.1x_2^2$, com t = 0.7 (t <  0.9090)

In [11]:
a = np.float64([1,1.1]) # temos, por exemplo, esses valores de a > 0
x = np.float64([1,2]) # e um chute inicial dos valores de xi

convergence_x = gradient_descent(x, a, step_size=0.7, backtracking=False, tolerance=0.001, verbose=True)
print("Ponto de mínimo encontrado: ", convergence_x)

Iteracoes utilizadas =  14
Ponto de mínimo encontrado:  [2.68435456e-06 3.58544144e-04]


Função: $f(x_1, x_2) = 1x_1^2 + 100x_2^2$, com t = 0.007 (t <  0.01)

In [12]:
a = np.float64([1,100]) # temos, por exemplo, esses valores de a > 0
x = np.float64([1,2]) # e um chute inicial dos valores de xi

convergence_x = gradient_descent(x, a, step_size=0.007, backtracking=False, tolerance=0.001, verbose=True)
print("Ponto de mínimo encontrado: ", convergence_x)

Iteracoes utilizadas =  540
Ponto de mínimo encontrado:  [4.93780651e-004 2.59074884e-215]


**Análise:** Para a primeira função, podemos observar que o $t$ escolhido arbitrariamente resultou em mais iterações para realizar a convergência. Dessa forma, podemos dizer que há fortes indicativos de que o $t_{ótimo}$ performa bem para este problema.

Já para a segunda função, observamos que para o t arbitrário $t=0.007$ foram necessárias menos iterações do que com o $t_{ótimo}$ desta função. Este comportamento acontece pois os autovalores da função são mais discrepantes e assim se parecem mais com elipses achatadas, fazendo com que o $t_{ótimo}$ talvez não seja ideal para o problema, uma vez que o $t_{ótimo}$ irá realizar passos muito pequenos devido ao gradiente da função elíptica achatada e, assim, nosso $t=0.007$ obteve convergência em menos iterações por permitir a realização de passos maiores, acelerando o processo de convergência.

Ou seja, apesar de $2/(L+\mu)$ garantir a melhor taxa de convergência ($\frac{L-\mu}{L+\mu}$), para resultados práticos outros valores de $t$ podem convergir mais rapidamente, como demonstrado pela segunda função aqui utilizada.

**iii) Técnica de Busca Linear (backtracking)**

Função: $f(x_1, x_2) = 1x_1^2 + 1.1x_2^2$, com $t_{inicial} = 2$

In [13]:
a = np.float64([1,1.1]) # temos, por exemplo, esses valores de a > 0
x = np.float64([1,2]) # e um chute inicial dos valores de xi

convergence_x = gradient_descent(x, a, step_size=2, backtracking=True, tolerance=0.001, alpha=0.1, verbose=True)
print("Ponto de mínimo encontrado: ", convergence_x)

Iteracoes utilizadas =  4
Ponto de mínimo encontrado:  [0.     0.0002]


Função: $f(x_1, x_2) = 1x_1^2 + 100x_2^2$, com $t_{inicial} = 2$

In [14]:
a = np.float64([1,100]) # temos, por exemplo, esses valores de a > 0
x = np.float64([1,2]) # e um chute inicial dos valores de xi

convergence_x = gradient_descent(x, a, step_size=10, backtracking=True, tolerance=0.001, alpha=0.1, verbose=True)
print("Ponto de mínimo encontrado: ", convergence_x)

Iteracoes utilizadas =  355
Ponto de mínimo encontrado:  [2.86961111e-04 4.06313933e-06]


**Análise:** Para a primeira função, temos que a convergência se dá com um número semelhante de iterações ao método de passo fixo, indicando pouca diferença entre as técnicas utilizadas nesse caso (quando os autovalores são próximos): o passo fixo com o $t_{ótimo}$ convergiu de maneira um pouco mais rápida do que o *backtracking*, que obteve convergência com uma iteração a mais, mas com a vantagem de não ser necessário se preocupar tanto com o valor de $t$ utilizado.

Já para a segunda função (autovalores discrepantes) observa-se grande ganho ao utilizar a técnica de *busca linear (backtracking)*, uma vez que esta convergiu com 355 iterações, contra 645 iterações do $t_{ótimo}$ e 540 iterações do $t_{arbitrário}$, com a vantagem de não ter que se preocupar tanto com o valor de $t$, uma vez que este valor é adaptado ao longo das iterações.

### Testes para diferentes valores de $n$

Aqui vamos padronizar os testes para o ponto inicial $x_{inicial}$ ser $(1,1,...,1)$. Para facilitar a automatização dos experimentos, criamos funções auxiliares:

In [15]:
initial_x = lambda x: [1 for i in range(x)]
get_vec = lambda n,comparison_val: np.linspace(1.0, comparison_val, num=n)

In [16]:
import time

def max_n_2_min(max_n, backtracking, step_size):
    times = []
    final_time = 0
    n = 2

    while(final_time < 120): # Converged in less than 2 minutes
        # Create variables
        a = np.array(get_vec(n, max_n))
        x = np.array(initial_x(n))
        
        # Run and compute time taken
        initial_time = time.clock()
        convergence_x = gradient_descent(x, a, step_size=step_size, backtracking=backtracking, tolerance=0.001, alpha=0.1)
        final_time = time.clock() - initial_time
        times.append(final_time)
        n = n * 2
    
    print("final time{}".format(final_time))
    return int(n/2) # last n

Função com $n$ elementos, com maior valor 1.1 vezes maior que o menor, utilizando backtracking. Passo inicial t = 10.

In [17]:
n = max_n_2_min(1.1, backtracking = True, step_size = 10)
print("O n convergiu dentro de aproximadamente 2 minutos para n até ", n)

O n convergiu dentro de aproximadamente 2 minutos para n até  67108864


Função com $n$ elementos, com maior valor 100 vezes maior que o menor, utilizando backtracking. Passo inicial t = 10.

In [18]:
n = max_n_2_min(100, backtracking = True, step_size = 10)
print("O n convergiu dentro de aproximadamente 2 minutos para n até ", n)

O n convergiu dentro de aproximadamente 2 minutos para n até  1048576


Função com $n$ elementos, com maior valor 1.1 vezes maior que o menor, utilizando **passo fixo** com t = $2/(\mu + L)$

In [19]:
n = max_n_2_min(1.1, backtracking = False, step_size = optimal_step_1)
print("O n convergiu dentro de aproximadamente 2 minutos para n até ", n)

O n convergiu dentro de aproximadamente 2 minutos para n até  268435456


Função com $n$ elementos, com maior valor 100 vezes maior que o menor, utilizando **passo fixo** com t = $2/(\mu + L)$

In [20]:
n = max_n_2_min(100, backtracking = False, step_size = optimal_step_2)
print("O n convergiu dentro de aproximadamente 2 minutos para n até ", n)

O n convergiu dentro de aproximadamente 2 minutos para n até  8388608


**Análise:** Pode-se observar que o método gradiente utilizando *backtracking* converge mais rápido para a função onde o maior valor de $a_i$ é 1.1 vezes o menor valor de $a_i$ do que para a função com maior valor de $a_i$ sendo 100 vezes o menor valor de $a_i$. Desta forma, podemos concluir que há indícios de que funções com autovalores mais discrepantes são mais custosas computacionalmente, ou seja, sua complexidade assintótica é maior.

Adicionalmente, os últimos dois experimentos retratam o mesmo procedimento só que para o método do passo fixo. É possível observar que, assim como anteriormente, a função com autovalores mais próximos possui um tempo de execução menor, permitindo assim a execução de funções com n maior do que a função com os autovalores mais discrepantes no tempo de 2 minutos.

Por fim, nota-se que o *passo fixo* consegue executar uma carga maior de trabalho com menos esforço. Isso indica que os custos computacionais associados a calcular as adaptações de passo no *backtracking* é alto para valores de $n$ grandes, enquanto o passo fixo acaba ficando mais vantajoso nessa situação já o cálculo do passo ótimo é simples para estas funções e seu resultado é utilizado independentemente do tamanho da entrada. Em particular, essa diferença é ainda mais notável para autovalores mais distantes.